<a href="https://colab.research.google.com/github/maragraziani/interpretAI_DigiPath/blob/main/concept_attribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
EXPERIMENT_TYPE = 'RCV_VGG_PCAM'

# Concept attribution
... description

In [2]:
!pip install tensorflow==1.8.0
!pip install keras==2.1.6
!pip install rcvtool
import keras
import sys
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD, Adadelta
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras import metrics
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import time
import sys
import shutil
import numpy as np
from keras.preprocessing import image
import cv2

     |████████████████████████████████| 49.1MB 86kB/s 
     |████████████████████████████████| 3.1MB 40.4MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107222 sha256=b751ec5ac3cb4490cb995e1f1ae412d9d94f7f953c66e131baa03f1d78602761
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.1
    Uninstalling bleach-3.2.1:
      Successfully uninstalled bleach-3.2.1
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0
     |███████████████████████

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [3]:
keras.backend.clear_session()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import numpy as np
global new_folder
folder_name=EXPERIMENT_TYPE
new_folder = './'+ folder_name #new_folder
if not os.path.exists(new_folder):
    os.mkdir(new_folder)
global error_log
error_log=open(new_folder+'/ERR.log', 'w')
BATCH_SIZE = 32

# SAVE FOLD
f=open(new_folder+"/seed.txt","w")
seed=0
print(seed)
f.write(str(seed))
f.close()

# SET SEED
np.random.seed(seed)
#tf.set_random_seed(seed)

0


In [6]:
# STAIN NORMALIZATION FUNCTIONS
def get_normalizer(patch, save_folder=''):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    print('Normalisers saved to disk.')
    return normalizer

def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))
    
# BATCH GENERATORS
import sys
sys.path.append('/content/drive/MyDrive/CNNinterpret/')
from normalizers import *
global normalizer
normalizer=get_normalizer(np.load('/content/drive/MyDrive/CNNinterpret/normalizing_patch.npy'))
def get_batch_data(patch_list, labels, batch_size=32):
    num_samples=len(patch_list)
    while True:
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            batch_samples=patch_list[offset:offset+batch_size]
            for patch_id in range(len(batch_samples)):
                #print(len(batch_samples))
                patch=patch_list[patch_id]
                patch=normalize_patch(patch, normalizer)
                # VGG input is fixed to 224,224 for finetuning imagenet weights
                patch=cv2.resize(patch, (224,224)) 
                patch=keras.applications.vgg16.preprocess_input(patch)
                label=labels[patch_id]
                batch_x.append(patch)
                batch_y.append(label)
            
            batch_x=np.asarray(batch_x, dtype=np.float32)/255.
            batch_y=np.asarray(batch_y, dtype=np.float32)
            yield batch_x, batch_y

Using brightness standardization
Normalisers saved to disk.


In [7]:
!wget https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz
!mv camelyonpatch_level_2_split_valid_x.h5.gz /content/drive/MyDrive/CNNinterpret/
!gunzip /content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_x.h5.gz


--2020-12-21 13:54:41--  https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 805965320 (769M) [application/octet-stream]
Saving to: ‘camelyonpatch_level_2_split_valid_x.h5.gz’

camelyonpatch_level 100%[===================>] 768.63M  17.0MB/s    in 1m 50s  

2020-12-21 13:56:33 (6.99 MB/s) - ‘camelyonpatch_level_2_split_valid_x.h5.gz’ saved [805965320/805965320]



In [8]:
import h5py as hd
y_test = np.asarray(hd.File('/content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_y.h5', 'r')['y'][:])
x_test = np.asarray(hd.File('/content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_x.h5', 'r')['x'][:])
y_test=y_test[:].ravel()
test_generator = get_batch_data(x_test, y_test)
x_b, y_b = test_generator.__next__()

In [9]:
!wget https://zenodo.org/record/4322725/files/weights.h5
!mv weights.h5 /content/drive/MyDrive/CNNinterpret/

--2020-12-21 13:57:09--  https://zenodo.org/record/4322725/files/weights.h5
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 537118144 (512M) [application/octet-stream]
Saving to: ‘weights.h5’

weights.h5          100%[===================>] 512.24M  19.7MB/s    in 28s     

2020-12-21 13:57:38 (18.6 MB/s) - ‘weights.h5’ saved [537118144/537118144]



In [ ]:
#Let's load the VGG weights finetuned on the histopathology dataset.
base_model=keras.applications.VGG16(weights='imagenet')
predictions=keras.layers.Dense(1,activation='sigmoid')(base_model.layers[-2].output)
vgg_histo_model = keras.Model(base_model.input, predictions)
vgg_histo_model.load_weights('/content/drive/MyDrive/CNNinterpret/weights.h5')
vgg_histo_model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=1e-4), metrics=['accuracy'])

496271360/553467096 [=========================>....] - ETA: 1s

In [ ]:
# Let's extract the concept measures
import rcvtool as rcv


cm=[]
for x in x_b:
  cm.append(rcv.get_texture_measure(x_b, mtype='ASM'))
